# Pose Preprocessing with MediaPipe Tasks API

This notebook extracts pose landmarks from exercise videos using the **new MediaPipe Tasks API**.

## Prerequisites

Ensure you have the correct package versions installed:

```bash
pip install tensorflow==2.20.0 mediapipe==0.10.31 protobuf>=5.28.0
```

Then restart the kernel before running the cells below.

In [2]:
import sys
import os
import logging
import numpy as np
from pathlib import Path

# Setup paths
project_root = Path.cwd().parent.parent

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print(f"Project root: {project_root}")

Project root: /mnt/d/Graduation_Project/ai-virtual-coach


## Download MediaPipe Model (First Time Only)

The new MediaPipe Tasks API requires a model file. We're using the **full** model for better accuracy.

In [3]:
import urllib.request
from pathlib import Path

# Model download URL - Using FULL model for better accuracy
MODEL_URL = 'https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_full/float16/latest/pose_landmarker_full.task'
MODEL_PATH = project_root / 'datasets' / 'pose_landmarker_full.task'

# Download if not exists
if not MODEL_PATH.exists():
    print(f"Downloading MediaPipe pose model (FULL) to {MODEL_PATH}...")
    MODEL_PATH.parent.mkdir(parents=True, exist_ok=True)
    urllib.request.urlretrieve(MODEL_URL, MODEL_PATH)
    print(f"✅ Model downloaded successfully ({MODEL_PATH.stat().st_size / 1024 / 1024:.1f} MB)")
else:
    print(f"✅ Model already exists at {MODEL_PATH} ({MODEL_PATH.stat().st_size / 1024 / 1024:.1f} MB)")

# NOW import the preprocessing module (after model is available)
print("\nImporting preprocessing module...")
sys.path.insert(0, str(project_root / 'src'))
from preprocessing.preprocess_pose_RGB import extract_pose_estimates
print("✅ Module imported successfully!")

✅ Model downloaded successfully (9.0 MB)

Importing preprocessing module...
✅ Module imported successfully!


## Configuration

In [5]:
# Paths
CLIPS_PATH = project_root / 'datasets' / 'Clips'
OUTPUT_DIR = project_root / 'datasets' / 'Mediapipe pose estimates'

# Create output directory if it doesn't exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Parameters
T_FIXED = 80  # Fixed length for temporal sequences
VIEWS = ['front', 'side']  # Views to process

print(f"Clips directory: {CLIPS_PATH}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Fixed temporal length: {T_FIXED} frames")

Clips directory: /mnt/d/Graduation_Project/ai-virtual-coach/datasets/Clips
Output directory: /mnt/d/Graduation_Project/ai-virtual-coach/datasets/Mediapipe pose estimates
Fixed temporal length: 80 frames


## Process Front View

In [ ]:
print("="*60)
print("PROCESSING FRONT VIEW")
print("="*60)

front_output_path = OUTPUT_DIR / 'pose_data_front.npz'

front_dataset, front_stats, front_failed = extract_pose_estimates(
    clips_path=str(CLIPS_PATH),
    view='front',
    T_fixed=T_FIXED,
    output_path=str(front_output_path)
)

print("\n" + "="*60)
print("FRONT VIEW SUMMARY")
print("="*60)
print(f"Total reps extracted: {front_stats['total_reps']}")
print(f"Unique subjects: {front_stats['unique_subjects']}")
print(f"Unique exercises: {front_stats['unique_exercises']}")
print(f"Videos processed: {front_stats['total_videos_processed']}")
print(f"Total frames extracted: {front_stats['total_frames_extracted']}")
print(f"Failed videos: {front_stats['failed_videos']}")
print(f"\nStatic features shape: {front_dataset['X_static'].shape}")
print(f"Temporal features shape: {front_dataset['X_temporal'].shape}")
print(f"\n⏱️ Tempo Statistics:")
print(f"  Duration (mean/median): {front_stats['tempo_stats']['duration_mean']:.2f}s / {front_stats['tempo_stats']['duration_median']:.2f}s")
print(f"  Frame count (mean/median): {front_stats['tempo_stats']['frame_count_mean']:.1f} / {front_stats['tempo_stats']['frame_count_median']:.0f}")
print(f"  FPS values: {front_stats['tempo_stats']['fps_unique']}")
print(f"\n📁 Output Files:")
print(f"  Static:   {front_stats.get('static_file', 'N/A')}")
print(f"  Temporal: {front_stats.get('temporal_file', 'N/A')}")

INFO - Scanning clips directory for front view...


PROCESSING FRONT VIEW


Scanning front videos: 100%|██████████| 15/15 [00:20<00:00,  1.40s/it]
INFO - [scan_clips_directory] Scanned front view: 147 samples, 1574 videos, 49 subjects, 15 exercises
INFO - Found 147 sample(s) to process
Extracting front poses:   0%|          | 0/147 [00:00<?, ?it/s]INFO - 
Processing: Dumbbell shoulder press / volunteer_001
W0000 00:00:1767722500.867032  623302 landmark_projection_calculator.cc:78] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
INFO -   ✓ Extracted 11 rep(s)
Extracting front poses:   1%|          | 1/147 [00:21<51:57, 21.35s/it]INFO - 
Processing: Dumbbell shoulder press / volunteer_010
INFO -   ✓ Extracted 10 rep(s)
Extracting front poses:   1%|▏         | 2/147 [00:44<53:35, 22.18s/it]INFO - 
Processing: Dumbbell shoulder press / volunteer_002
INFO -   ✓ Extracted 10 rep(s)
Extracting front poses:   2%|▏         | 3/147 [01:10<58:17, 24.29s/it]INFO - 
Processing: Dumbbell shoul

### Failed Videos (Front)

In [ ]:
if front_failed:
    print(f"\n⚠️ {len(front_failed)} samples failed to process:")
    for item in front_failed[:10]:  # Show first 10
        print(f"  - {item['exercise']} / {item['subject']}: {item['error']}")
    if len(front_failed) > 10:
        print(f"  ... and {len(front_failed) - 10} more")
else:
    print("✅ All front view samples processed successfully!")

✅ All front view samples processed successfully!


## Process Side View

In [ ]:
print("="*60)
print("PROCESSING SIDE VIEW")
print("="*60)

side_output_path = OUTPUT_DIR / 'pose_data_side.npz'

side_dataset, side_stats, side_failed = extract_pose_estimates(
    clips_path=str(CLIPS_PATH),
    view='side',
    T_fixed=T_FIXED,
    output_path=str(side_output_path)
)

print("\n" + "="*60)
print("SIDE VIEW SUMMARY")
print("="*60)
print(f"Total reps extracted: {side_stats['total_reps']}")
print(f"Unique subjects: {side_stats['unique_subjects']}")
print(f"Unique exercises: {side_stats['unique_exercises']}")
print(f"Videos processed: {side_stats['total_videos_processed']}")
print(f"Total frames extracted: {side_stats['total_frames_extracted']}")
print(f"Failed videos: {side_stats['failed_videos']}")
print(f"\nStatic features shape: {side_dataset['X_static'].shape}")
print(f"Temporal features shape: {side_dataset['X_temporal'].shape}")
print(f"\n⏱️ Tempo Statistics:")
print(f"  Duration (mean/median): {side_stats['tempo_stats']['duration_mean']:.2f}s / {side_stats['tempo_stats']['duration_median']:.2f}s")
print(f"  Frame count (mean/median): {side_stats['tempo_stats']['frame_count_mean']:.1f} / {side_stats['tempo_stats']['frame_count_median']:.0f}")
print(f"  FPS values: {side_stats['tempo_stats']['fps_unique']}")
print(f"\n📁 Output Files:")
print(f"  Static:   {side_stats.get('static_file', 'N/A')}")
print(f"  Temporal: {side_stats.get('temporal_file', 'N/A')}")

INFO - Scanning clips directory for side view...


PROCESSING SIDE VIEW


Scanning side videos: 100%|██████████| 15/15 [00:04<00:00,  3.40it/s]
INFO - [scan_clips_directory] Scanned side view: 149 samples, 1571 videos, 49 subjects, 15 exercises
INFO - Found 149 sample(s) to process
Extracting side poses:   0%|          | 0/149 [00:00<?, ?it/s]INFO - 
Processing: Dumbbell shoulder press / volunteer_001
INFO -   ✓ Extracted 11 rep(s)
Extracting side poses:   1%|          | 1/149 [00:13<32:49, 13.31s/it]INFO - 
Processing: Dumbbell shoulder press / volunteer_010
INFO -   ✓ Extracted 10 rep(s)
Extracting side poses:   1%|▏         | 2/149 [00:28<35:45, 14.60s/it]INFO - 
Processing: Dumbbell shoulder press / volunteer_002
INFO -   ✓ Extracted 10 rep(s)
Extracting side poses:   2%|▏         | 3/149 [00:39<31:33, 12.97s/it]INFO - 
Processing: Dumbbell shoulder press / volunteer_003
INFO -   ✓ Extracted 11 rep(s)
Extracting side poses:   3%|▎         | 4/149 [00:51<30:28, 12.61s/it]INFO - 
Processing: Dumbbell shoulder press / volunteer_031
INFO -   ✓ Extracted 8 re


SIDE VIEW SUMMARY
Total reps extracted: 1571
Unique subjects: 49
Unique exercises: 15
Videos processed: 1571
Total frames extracted: 127387
Failed videos: 0

Static features shape: (1571, 45)
Temporal features shape: (1571, 50, 9)

📁 Output Files:
  Static:   /mnt/d/Graduation_Project/ai-virtual-coach/datasets/Mediapipe pose estimates/pose_data_side_static.npz
  Temporal: /mnt/d/Graduation_Project/ai-virtual-coach/datasets/Mediapipe pose estimates/pose_data_side_temporal.npz


### Failed Videos (Side)

In [ ]:
if side_failed:
    print(f"\n⚠️ {len(side_failed)} samples failed to process:")
    for item in side_failed[:10]:  # Show first 10
        print(f"  - {item['exercise']} / {item['subject']}: {item['error']}")
    if len(side_failed) > 10:
        print(f"  ... and {len(side_failed) - 10} more")
else:
    print("✅ All side view samples processed successfully!")

✅ All side view samples processed successfully!


## Combined Summary

In [ ]:
print("="*60)
print("COMBINED SUMMARY (FRONT + SIDE)")
print("="*60)

total_reps = front_stats['total_reps'] + side_stats['total_reps']
total_videos = front_stats['total_videos_processed'] + side_stats['total_videos_processed']
total_frames = front_stats['total_frames_extracted'] + side_stats['total_frames_extracted']
total_failed = front_stats['failed_videos'] + side_stats['failed_videos']

# Combined unique counts
all_subjects = set(front_dataset['subject_ids']) | set(side_dataset['subject_ids'])
all_exercises = set(front_dataset['exercise_names']) | set(side_dataset['exercise_names'])

print(f"\n📊 Dataset Statistics:")
print(f"  Total reps: {total_reps}")
print(f"  Total videos: {total_videos}")
print(f"  Total frames: {total_frames}")
print(f"  Unique volunteers: {len(all_subjects)}")
print(f"  Unique exercises: {len(all_exercises)}")
print(f"  Failed samples: {total_failed}")

print(f"\n📁 Output Files:")
print(f"  Front Static:   {front_stats.get('static_file', 'N/A')}")
print(f"  Front Temporal: {front_stats.get('temporal_file', 'N/A')}")
print(f"  Side Static:    {side_stats.get('static_file', 'N/A')}")
print(f"  Side Temporal:  {side_stats.get('temporal_file', 'N/A')}")

print(f"\n✅ Preprocessing complete!")

COMBINED SUMMARY (FRONT + SIDE)

📊 Dataset Statistics:
  Total reps: 3145
  Total videos: 3145
  Total frames: 264471
  Unique volunteers: 49
  Unique exercises: 15
  Failed samples: 0

📁 Output Files:
  Front Static:   /mnt/d/Graduation_Project/ai-virtual-coach/datasets/Mediapipe pose estimates/pose_data_front_static.npz
  Front Temporal: /mnt/d/Graduation_Project/ai-virtual-coach/datasets/Mediapipe pose estimates/pose_data_front_temporal.npz
  Side Static:    /mnt/d/Graduation_Project/ai-virtual-coach/datasets/Mediapipe pose estimates/pose_data_side_static.npz
  Side Temporal:  /mnt/d/Graduation_Project/ai-virtual-coach/datasets/Mediapipe pose estimates/pose_data_side_temporal.npz

✅ Preprocessing complete!


## Verify Output Files

In [ ]:
# Load and inspect saved files
print("Verifying saved NPZ files...\n")

for view in ['front', 'side']:
    stats = front_stats if view == 'front' else side_stats
    
    # Verify static file
    static_file = stats.get('static_file')
    if static_file and os.path.exists(static_file):
        print(f"\n{view.upper()} VIEW - STATIC:")
        data = np.load(static_file, allow_pickle=True)
        print(f"  File: {os.path.basename(static_file)}")
        print(f"  Keys: {list(data.keys())}")
        print(f"  X_static: {data['X_static'].shape} - {data['X_static'].dtype}")
        print(f"  exercise_names: {data['exercise_names'].shape}")
        print(f"  subject_ids: {data['subject_ids'].shape}")
        print(f"  tempo_duration_sec: {data['tempo_duration_sec'].shape}")
        print(f"  tempo_frame_count: {data['tempo_frame_count'].shape}")
        print(f"  tempo_fps: {data['tempo_fps'].shape}")
        print(f"  view: {data['view']}")
        print(f"  angle_names: {list(data['angle_names'])}")
    
    # Verify temporal file
    temporal_file = stats.get('temporal_file')
    if temporal_file and os.path.exists(temporal_file):
        print(f"\n{view.upper()} VIEW - TEMPORAL:")
        data = np.load(temporal_file, allow_pickle=True)
        print(f"  File: {os.path.basename(temporal_file)}")
        print(f"  Keys: {list(data.keys())}")
        print(f"  X_temporal: {data['X_temporal'].shape} - {data['X_temporal'].dtype}")
        print(f"  exercise_names: {data['exercise_names'].shape}")
        print(f"  subject_ids: {data['subject_ids'].shape}")
        print(f"  tempo_duration_sec: {data['tempo_duration_sec'].shape}")
        print(f"  tempo_frame_count: {data['tempo_frame_count'].shape}")
        print(f"  tempo_fps: {data['tempo_fps'].shape}")
        print(f"  view: {data['view']}")
        print(f"  T_fixed: {data['T_fixed']}")
        print(f"  angle_names: {list(data['angle_names'])}")

print("\n✅ All files verified successfully!")

Verifying saved NPZ files...


FRONT VIEW - STATIC:
  File: pose_data_front_static.npz
  Keys: ['X_static', 'exercise_names', 'subject_ids', 'view', 'angle_names']
  X_static: (1574, 45) - float32
  exercise_names: (1574,)
  subject_ids: (1574,)
  view: front
  angle_names: ['left_elbow', 'right_elbow', 'left_shoulder', 'right_shoulder', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'torso_lean']

FRONT VIEW - TEMPORAL:
  File: pose_data_front_temporal.npz
  Keys: ['X_temporal', 'exercise_names', 'subject_ids', 'view', 'T_fixed', 'angle_names']
  X_temporal: (1574, 50, 9) - float32
  exercise_names: (1574,)
  subject_ids: (1574,)
  view: front
  T_fixed: 50
  angle_names: ['left_elbow', 'right_elbow', 'left_shoulder', 'right_shoulder', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'torso_lean']

SIDE VIEW - STATIC:
  File: pose_data_side_static.npz
  Keys: ['X_static', 'exercise_names', 'subject_ids', 'view', 'angle_names']
  X_static: (1571, 45) - float32
  exercise_names:

## Sample Data Inspection

In [ ]:
# Inspect first few samples
print("Sample data from front view:\n")
print("First 5 exercises:")
for i in range(min(5, len(front_dataset['exercise_names']))):
    print(f"  {i+1}. {front_dataset['exercise_names'][i]} (Subject {front_dataset['subject_ids'][i]})")

print("\nStatic features (first rep, first 10 values):")
print(front_dataset['X_static'][0, :10])

print("\nTemporal features (first rep, first 5 frames):")
print(front_dataset['X_temporal'][0, :5, :])

Sample data from front view:

First 5 exercises:
  1. Dumbbell shoulder press (Subject 1)
  2. Dumbbell shoulder press (Subject 1)
  3. Dumbbell shoulder press (Subject 1)
  4. Dumbbell shoulder press (Subject 1)
  5. Dumbbell shoulder press (Subject 1)

Static features (first rep, first 10 values):
[ 65.43624   31.717201  28.484045 118.76366   90.27961   71.211555
  32.349236  33.099964 124.33251   91.232544]

Temporal features (first rep, first 5 frames):
[[ 30.85594    33.65273    81.67808    80.56241   164.48721   161.15627
  173.01715   171.40184     2.0278711]
 [ 30.871962   34.592854   81.11283    80.96457   164.64124   160.59013
  172.05402   172.79015     1.6828606]
 [ 30.744637   34.657204   81.031685   80.89822   164.55688   160.64856
  172.0019    172.84607     1.6818085]
 [ 30.64689    34.707226   80.94014    80.8559    164.36314   160.74544
  171.86783   172.89949     1.6647671]
 [ 30.562447   34.712177   80.7624     80.82227   164.09644   160.85959
  171.71594   172.9682